In [1]:
import numpy as np
import re
from operator import itemgetter

In [2]:
Corpus = {
    'l o w _':5,
    'l o w e r _':2,
    'n e w e s t _':6,
    'w i d e s t _':3,
    'h a p p i e r _':2
}

In [3]:
def getPairCounts(Corpus):
    pairs = {}
    for word, fr in Corpus.items(): #fr = freequency
        symbols = word.split(' ')
        for i in range(len(symbols)-1):
            pair = (symbols[i], symbols[i+1])
            cfr = pairs.get(pair, 0) #cfr = current freequency
            pairs[pair] = cfr + fr
    return pairs

In [4]:
pairCounts = getPairCounts(Corpus)

In [5]:
pairCounts

{('l', 'o'): 7,
 ('o', 'w'): 7,
 ('w', '_'): 5,
 ('w', 'e'): 8,
 ('e', 'r'): 4,
 ('r', '_'): 4,
 ('n', 'e'): 6,
 ('e', 'w'): 6,
 ('e', 's'): 9,
 ('s', 't'): 9,
 ('t', '_'): 9,
 ('w', 'i'): 3,
 ('i', 'd'): 3,
 ('d', 'e'): 3,
 ('h', 'a'): 2,
 ('a', 'p'): 2,
 ('p', 'p'): 2,
 ('p', 'i'): 2,
 ('i', 'e'): 2}